# Monitor performance and input data in a deployed webservice

Learn how to instrument your scoring scripts to log debugging output and model payloads, while also sending performance metrics to Application Insights.

Then, feed in simulated data to learn how drift detection works.


In [80]:
## Get workspace

In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='default', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='azureml-examples')

## Get path to root of repo

In [15]:
import git
from pathlib import Path

# get root of git repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

## Get compute target

In [16]:
from azureml.core.compute import AksCompute

#AksCompute.list(workspace=ws)
compute = AksCompute(ws, 'aks-cpu-deploy')

## Deploy webservice with logging enabled

In [164]:
from azureml.core.model import InferenceConfig, Model
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
from azureml.core.webservice import AksWebservice, LocalWebservice

#env = Environment.get(ws, 'AzureML-Minimal')
env = Environment(name="myenv")

conda_dep = CondaDependencies()
packages_to_add = ['schedule', 'azure-storage-blob']
for package in packages_to_add:
    conda_dep.add_pip_package(package)

env.python.conda_dependencies = conda_dep

dstore = ws.get_default_datastore()
env.environment_variables['DS_CONNECTION_STRING'] = \
    'DefaultEndpointsProtocol=' + dstore.protocol + \
    ';AccountName=' + dstore.account_name + ';AccountKey=' \
    + dstore.account_key + ';EndpointSuffix=' + dstore.endpoint

inf_config = InferenceConfig(
    environment=env,
    source_directory=prefix.joinpath("code", "deployment", "monitor"),
    entry_script="score.py")

deploy_config = AksWebservice.deploy_configuration()
test = LocalWebservice.deploy_configuration(port=6789)

#service = Model.deploy(name="monitor", models=[], workspace=ws, inference_config=inf_config, deployment_config=deploy_config, deployment_target=compute, overwrite=True)
service = Model.deploy(name="local-test", models=[], workspace=ws, inference_config=inf_config, deployment_config=test, overwrite=True)

service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 0e14976437204610b0f33e3f974544ac.azurecr.io
Logging into Docker registry 0e14976437204610b0f33e3f974544ac.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 0e14976437204610b0f33e3f974544ac.azurecr.io/azureml/azureml_4c46604794d900054fa72c5dc5b83410
 ---> 2d590c4e9b9a
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 85f57c448b18
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjY1NjA1NzVkLWZhMDYtNGU3ZC05NWZiLWY5NjJlNzRlZmQ3YSIsInJlc291cmNlR3JvdXBOYW1lIjoiYXp1cmVtbC1leGFtcGxlcyIsImFjY291bnROYW1lIjoiZGVmYXVsdCIsIndvcmtzcGFjZUlkIjoiMGUxNDk3NjQtMzcyMC00NjEwLWIwZjMtM2UzZjk3NDU0NGFjIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 0bcb0b37ef6c
 ---> bfa8fc6da87e
Step 4/5 : RUN mv '/var/azureml-app/tmpa9d06wmn.py' /var/azureml-app/main.py
 ---> Running in fa4d2f8dc11

WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [165]:
print(service.get_logs())

2020-10-23T21:09:54,760707900+00:00 - gunicorn/run 
2020-10-23T21:09:54,760804600+00:00 - rsyslog/run 
2020-10-23T21:09:54,762870000+00:00 - iot-server/run 
2020-10-23T21:09:54,763295300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_1d4ca3f5e2f41bbcb1793fb78e0a1acf/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1d4ca3f5e2f41bbcb1793fb78e0a1acf/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1d4ca3f5e2f41bbcb1793fb78e0a1acf/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1d4ca3f5e2f41bbcb1793fb78e0a1acf/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1d4ca3f5e2f41bbcb1793fb78e0a1acf/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [150]:
from datetime import date
number = 3

data = json.dumps([str(date.today()), str(number)])
service.run(data)

{}

## Update service to collect model data and enable Application Insights

In [157]:
inf_config = InferenceConfig(
    environment=env,
    source_directory=prefix.joinpath("code", "deployment", "monitor"),
    entry_script="score.py")


service.update(inference_config=inf_config, enable_app_insights=True, collect_model_data=True)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 0e14976437204610b0f33e3f974544ac.azurecr.io
Logging into Docker registry 0e14976437204610b0f33e3f974544ac.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 0e14976437204610b0f33e3f974544ac.azurecr.io/azureml/azureml_4c46604794d900054fa72c5dc5b83410
 ---> 2d590c4e9b9a
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> f3b4ae300805
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjY1NjA1NzVkLWZhMDYtNGU3ZC05NWZiLWY5NjJlNzRlZmQ3YSIsInJlc291cmNlR3JvdXBOYW1lIjoiYXp1cmVtbC1leGFtcGxlcyIsImFjY291bnROYW1lIjoiZGVmYXVsdCIsIndvcmtzcGFjZUlkIjoiMGUxNDk3NjQtMzcyMC00NjEwLWIwZjMtM2UzZjk3NDU0NGFjIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in bb94429ce434
 ---> b4af0cc3ae7d
Step 4/5 : RUN mv '/var/azureml-app/tmplzou33_x.py' /var/azureml-app/main.py
 ---> Running in acc99fa1a7d

## Create baseline data

In [152]:
import requests
from datetime import date
import json
import random

def feed_data_range(lower_bound, upper_bound, date):
    for i in range(100):
        number = random.randrange(lower_bound, upper_bound)
        data = json.dumps([str(date), str(number)])

        token = "Bearer" #f"Bearer {service.get_keys()[0]}"

        headers = {
            'Content-Type': 'application/json',
            'Authorization': token}
        print(f"posting request with data {data}")
        out = requests.post(service.scoring_uri, data=data, headers=headers)

feed_data_range(0, 5, date.today())

#service.run("30")

posting request with data ["2020-10-23", "3"]
posting request with data ["2020-10-23", "3"]
posting request with data ["2020-10-23", "4"]
posting request with data ["2020-10-23", "0"]
posting request with data ["2020-10-23", "0"]
posting request with data ["2020-10-23", "0"]
posting request with data ["2020-10-23", "3"]
posting request with data ["2020-10-23", "4"]
posting request with data ["2020-10-23", "2"]
posting request with data ["2020-10-23", "0"]
posting request with data ["2020-10-23", "4"]
posting request with data ["2020-10-23", "1"]
posting request with data ["2020-10-23", "1"]
posting request with data ["2020-10-23", "3"]
posting request with data ["2020-10-23", "2"]
posting request with data ["2020-10-23", "4"]
posting request with data ["2020-10-23", "0"]
posting request with data ["2020-10-23", "1"]
posting request with data ["2020-10-23", "3"]
posting request with data ["2020-10-23", "1"]
posting request with data ["2020-10-23", "4"]
posting request with data ["2020-1

## Create target data

In [155]:
from datetime import timedelta

feed_data_range(10, 15, date.today() - timedelta(days=1))

posting request with data ["2020-10-22", "12"]
posting request with data ["2020-10-22", "13"]
posting request with data ["2020-10-22", "11"]
posting request with data ["2020-10-22", "11"]
posting request with data ["2020-10-22", "12"]
posting request with data ["2020-10-22", "11"]
posting request with data ["2020-10-22", "12"]
posting request with data ["2020-10-22", "10"]
posting request with data ["2020-10-22", "13"]
posting request with data ["2020-10-22", "11"]
posting request with data ["2020-10-22", "11"]
posting request with data ["2020-10-22", "10"]
posting request with data ["2020-10-22", "10"]
posting request with data ["2020-10-22", "10"]
posting request with data ["2020-10-22", "12"]
posting request with data ["2020-10-22", "13"]
posting request with data ["2020-10-22", "14"]
posting request with data ["2020-10-22", "14"]
posting request with data ["2020-10-22", "12"]
posting request with data ["2020-10-22", "10"]
posting request with data ["2020-10-22", "11"]
posting reque

In [96]:
print(service.get_logs())

plication/json
2020-10-23 04:59:52,876 | root | INFO | 	Accept-Encoding: gzip, deflate
	Accept-Encoding: gzip, deflate
2020-10-23 04:59:52,876 | root | INFO | 	Accept: */*
	Accept: */*
2020-10-23 04:59:52,876 | root | INFO | 	X-Ms-Request-Id: fc77bc6a-85a4-4932-8b09-40da53e3baa6
	X-Ms-Request-Id: fc77bc6a-85a4-4932-8b09-40da53e3baa6
2020-10-23 04:59:52,876 | root | INFO | Scoring Timer is set to 30.0 seconds
Scoring Timer is set to 30.0 seconds
Received ["2020-10-23", "12"]. Should show up in App Insights logs
2020-10-23 04:59:52,877 | root | INFO | 200
200
127.0.0.1 - - [23/Oct/2020:04:59:52 +0000] "POST /score HTTP/1.0" 200 2 "-" "python-requests/2.24.0"
2020-10-23 04:59:53,858 | root | INFO | Validation Request Content-Type
Validation Request Content-Type
2020-10-23 04:59:53,859 | root | INFO | 	Host: 40.76.146.87
	Host: 40.76.146.87
2020-10-23 04:59:53,859 | root | INFO | 	X-Real-Ip: 10.240.1.12
	X-Real-Ip: 10.240.1.12
2020-10-23 04:59:53,859 | root | INFO | 	X-Forwarded-For: 10.24